# Reading CSV with Standard Library

Python provides a module in its standard library for reading and writing CSV or other delimited files.  It can be tempting to create or read such files using only Python's powerful string manipulation functionality.  Indeed, the author of this tutorial has done so far more often than he wishes to admit; however, it is a mistake to eschew the `csv` module which simply deals with many edge cases that are easy to overlook in quick scripts.

Let us start out by loading a few Python standard library modules that this lesson will utilize.

In [1]:
import csv
from pprint import pprint
from collections import namedtuple
from decimal import Decimal

# Doing it Wrong

In Python, the string methods `.split()` and `.join()` do 90% of what we need to in working with CSV.  The problem is, they do not do the other 10%.  Let's try a naive approach that goes bad.

In [2]:
fields = ["Name", "Evaluation", "Rating", "Age"]
data = [
    ["Mia Johnson", "The movie was excellent", 9.5, 25],
    ["Liam Lopez", "Didn't really like it", 3.0, 35],
    ["Isabella Lee", "Wow! That was great", 8.0, 45]
]

This is unremarkable data about several movie evaluations.  Let us try to serialize it.

In [3]:
with open('data/movie.csv', 'w') as movie:
    try:
        print(",".join(fields), file=movie)
        for record in data:
            print(",".join(record), file=movie)
    except Exception as err:
        print(err)

sequence item 2: expected str instance, float found


It is easy to see what went wrong.  The `.join()` method needs only strings in the iterable argument.  We can fix that fairly easily.  Python knows how to *stringify* all its objects.

In [4]:
with open('data/movie.csv', 'w') as movie:
    try:
        print(",".join(fields), file=movie)
        for record in data:
            print(",".join(str(r) for r in record), file=movie)
    except Exception as err:
        print(err)

Success! At least for now. Perhaps we want to read it back as a list of dictionaries.

We need to read the header first to use as keys, then we can pull values from each corresponding position in later rows.

In [5]:
with open('data/movie.csv') as movie:
    newdata = []
    keys = next(movie).split(',') # Header
    for line in movie:
        newdata.append(dict(zip(keys, line.split(','))))
    
pprint(newdata)

[{'Age\n': '25\n',
  'Evaluation': 'The movie was excellent',
  'Name': 'Mia Johnson',
  'Rating': '9.5'},
 {'Age\n': '35\n',
  'Evaluation': "Didn't really like it",
  'Name': 'Liam Lopez',
  'Rating': '3.0'},
 {'Age\n': '45\n',
  'Evaluation': 'Wow! That was great',
  'Name': 'Isabella Lee',
  'Rating': '8.0'}]


We did *pretty well*.  However, the last field of header and data have a trailing newline chacter we do not really want.  We can strip that, but other problems still arise.

In [6]:
with open('data/movie.csv') as movie:
    newdata = []
    line = next(movie).rstrip()  # Header
    keys = line.split(',') 
    for line in movie:
        line = line.rstrip()
        newdata.append(dict(zip(keys, line.split(','))))
    
pprint(newdata)

[{'Age': '25',
  'Evaluation': 'The movie was excellent',
  'Name': 'Mia Johnson',
  'Rating': '9.5'},
 {'Age': '35',
  'Evaluation': "Didn't really like it",
  'Name': 'Liam Lopez',
  'Rating': '3.0'},
 {'Age': '45',
  'Evaluation': 'Wow! That was great',
  'Name': 'Isabella Lee',
  'Rating': '8.0'}]


Our ad hoc reader and writer behave well enough.  Let us add an additional record and try again.

In [7]:
new_eval = ["Olivia Gupta", "Meh, not my thing", 2.5, 55]
data.append(new_eval)

with open('data/movie.csv', 'w') as movie:
    try:
        print(",".join(fields), file=movie)
        for record in data:
            print(",".join(str(r) for r in record), file=movie)
    except Exception as err:
        print(err)

We can see that something is going to go wrong when a field can legitimately contain the delimiter.

In [8]:
!cat data/movie.csv

Name,Evaluation,Rating,Age
Mia Johnson,The movie was excellent,9.5,25
Liam Lopez,Didn't really like it,3.0,35
Isabella Lee,Wow! That was great,8.0,45
Olivia Gupta,Meh, not my thing,2.5,55


Let's use the idential ad hoc reader to read the data on disk again.

In [9]:
with open('data/movie.csv') as movie:
    newdata = []
    line = next(movie).rstrip()
    keys = line.split(',') # Header
    for line in movie:
        line = line.rstrip()
        newdata.append(dict(zip(keys, line.split(','))))

pprint(newdata)

[{'Age': '25',
  'Evaluation': 'The movie was excellent',
  'Name': 'Mia Johnson',
  'Rating': '9.5'},
 {'Age': '35',
  'Evaluation': "Didn't really like it",
  'Name': 'Liam Lopez',
  'Rating': '3.0'},
 {'Age': '45',
  'Evaluation': 'Wow! That was great',
  'Name': 'Isabella Lee',
  'Rating': '8.0'},
 {'Age': '2.5',
  'Evaluation': 'Meh',
  'Name': 'Olivia Gupta',
  'Rating': ' not my thing'}]


As written, nothing crashed.  But we also get data in the wrong fields sometimes. Another likely problem is handling embedded newlines in strings; a few other edge cases also occur. We could complicate matters with some additional code, and eventually get it right.  But the Python standard library does that for us.

# The `csv` Module

In the basic case, using the `csv` module gives us a largely file-like interface.  It merely handles a few things that can go wrong automatically.

In [10]:
with open('data/movie.csv', 'w') as fh:
    movie = csv.writer(fh, quoting=csv.QUOTE_MINIMAL)
    for record in [fields]+data:
        movie.writerow(record)
        
!cat data/movie.csv

Reading the data back is similar, with quoting and escaping handled properly.

In [11]:
with open('data/movie.csv') as fh:
    movie = csv.reader(fh)
    for record in movie:
        print(record)

['Name', 'Evaluation', 'Rating', 'Age']
['Mia Johnson', 'The movie was excellent', '9.5', '25']
['Liam Lopez', "Didn't really like it", '3.0', '35']
['Isabella Lee', 'Wow! That was great', '8.0', '45']
['Olivia Gupta', 'Meh, not my thing', '2.5', '55']


## Data Typing

Unlike some other tools, the standard library `csv` module makes little attempt to impose datatypes.  During writing, it will, of course, stringify objects that are not strings.  It usually leaves the decision of casting to other types up to the programmer.  

In [12]:
with open('data/movie.csv', 'w') as fh:
    movie = csv.writer(fh, quoting=csv.QUOTE_NONNUMERIC)
    for record in [fields]+data:
        movie.writerow(record)
        
!cat data/movie.csv

The `csv` module provides a limited option to quote all strings and to infer that anything unquoted is a number instead.  The numeric type used is always a floating point for this rule.  If you wish to read in an int, or a Decimal or Fraction, or another numeric type, you still need to write more custom code.

In [13]:
with open('data/movie.csv') as fh:
    movie = csv.reader(fh, quoting=csv.QUOTE_NONNUMERIC)
    for record in movie:
        print(record)

['Name', 'Evaluation', 'Rating', 'Age']
['Mia Johnson', 'The movie was excellent', 9.5, 25.0]
['Liam Lopez', "Didn't really like it", 3.0, 35.0]
['Isabella Lee', 'Wow! That was great', 8.0, 45.0]
['Olivia Gupta', 'Meh, not my thing', 2.5, 55.0]


Probably what we really want is to specify various data types for various columns.  In the example, Age is probably meant as in integer and Rating as a fractional number.  While we are customizing, perhaps a different collection type than a list is a more descriptive way to store records.

In [14]:
# Specify special types, string by default
types = {'Age': int, 'Rating': Decimal}

with open('data/movie.csv') as fh:
    newdata = []
    # Create a descriptive record for this data
    movie = csv.reader(fh)
    fields = next(movie)
    Movie = namedtuple("Movie", fields)
    for record in movie:
        # Cast each item to its needed datatype
        for pos, datum in enumerate(record):
            cast = types.get(fields[pos], str)
            record[pos] = cast(datum)
        newdata.append(Movie(*record))
        
pprint(newdata)

[Movie(Name='Mia Johnson', Evaluation='The movie was excellent', Rating=Decimal('9.5'), Age=25),
 Movie(Name='Liam Lopez', Evaluation="Didn't really like it", Rating=Decimal('3.0'), Age=35),
 Movie(Name='Isabella Lee', Evaluation='Wow! That was great', Rating=Decimal('8.0'), Age=45),
 Movie(Name='Olivia Gupta', Evaluation='Meh, not my thing', Rating=Decimal('2.5'), Age=55)]


## Records as Dictionaries

A namedtuple or a dataclass are useful Python standard library types for structured collections. The built-in standard type for doing that is a dictionary.  The `csv` module includes classes to make reading or writing dicts convenient.  These different types have pros and cons, but all are useful.  Dictionaries come with convenience classes to make similar code slightly shorter.

In [15]:
with open('data/movie.csv') as fh:
    movie = csv.DictReader(fh, quoting=csv.QUOTE_NONNUMERIC)
    for record in movie:
        print(record)

{'Name': 'Mia Johnson', 'Evaluation': 'The movie was excellent', 'Rating': 9.5, 'Age': 25.0}
{'Name': 'Liam Lopez', 'Evaluation': "Didn't really like it", 'Rating': 3.0, 'Age': 35.0}
{'Name': 'Isabella Lee', 'Evaluation': 'Wow! That was great', 'Rating': 8.0, 'Age': 45.0}
{'Name': 'Olivia Gupta', 'Evaluation': 'Meh, not my thing', 'Rating': 2.5, 'Age': 55.0}


Writing dictionaries back out to CSV is very similar.  Here we add a minor option, `newline=` in order to be able to write records with newlines in strings.  While we are doing that, let us also use a different delimiter to demonstrate that.

In [16]:
with open('data/movie.txt', 'w', newline='') as fh:
    fields = ['Name', 'Rating', 'Age']
    movie = csv.DictWriter(fh, fieldnames=fields, delimiter="|")
    movie.writeheader()
    movie.writerow({'Name': 'Mia\nJohnson', 'Rating': 9.5, 'Age': 25})
    movie.writerow({'Age': 35, 'Name': 'Liam Lopez'})
    movie.writerow({'Name': 'Isabella "Bella" Lee', 'Rating': 8.0, 'Age': 45})

!cat data/movie.txt


"Mia





Despite the slightly surprising newline inside a field, this will round-trip perfectly fine because of the quote.  The quotes inside one of the fields are also handle correctly.

In [17]:
with open('data/movie.txt', newline='') as fh:
    movie = csv.DictReader(fh, delimiter="|")
    for record in movie:
        print(record)

{'Name': 'Mia\nJohnson', 'Rating': '9.5', 'Age': '25'}
{'Name': 'Liam Lopez', 'Rating': '', 'Age': '35'}
{'Name': 'Isabella "Bella" Lee', 'Rating': '8.0', 'Age': '45'}
